# Evaluate snapshots and plot results

In [1]:
import os
import re
import subprocess
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-white')

GPUS = "0" #should be set
root = '/home/ubuntu/Amin-SSD/caffe/'#should be set
os.chdir(root)
MODEL_DIR = 'examples/okutama-detection-test/720x576/' #should be set
SNAP_DIR = 'models/VGGNet/okutama/SSD_720x576/' #should be set


SOLVER_FILE = MODEL_DIR+'solver.prototxt'


Get list of snapshots.

In [2]:
snapshots = os.listdir(SNAP_DIR)
snapshots = filter(lambda x: '.caffemodel' in x, snapshots) # only the .caffemodel files
snapshots.sort(key = lambda x: int(re.sub('\D','', x))) # sort by iteration

snapshots = snapshots[1::2]
print snapshots

['VGG_okutama_SSD_720x576_iter_1000.caffemodel', 'VGG_okutama_SSD_720x576_iter_2000.caffemodel', 'VGG_okutama_SSD_720x576_iter_3000.caffemodel', 'VGG_okutama_SSD_720x576_iter_4000.caffemodel', 'VGG_okutama_SSD_720x576_iter_5000.caffemodel', 'VGG_okutama_SSD_720x576_iter_6000.caffemodel', 'VGG_okutama_SSD_720x576_iter_7000.caffemodel', 'VGG_okutama_SSD_720x576_iter_8000.caffemodel']


In [ ]:
def plot_scores():
    scores = []
    iters = []
    for i, s in enumerate(snapshots):
        log_file = os.path.join(SNAP_DIR, s.split('.')[0] + ".log")
        if not os.path.exists(log_file):
            continue
        with open(log_file) as f:
            lines = f.readlines()
            iteration = [l for l in lines if "Finetuning from" in l]    
            score = [l for l in lines if "Test net output" in l] 
            if len(score)>0 and len(iteration)>0:
                iters.append(float(iteration[0].split("iter_")[1].split(".")[0]))
                scores.append(float(score[0].split('=')[1].strip()))

    plt.plot(iters, scores)
    plt.legend(["mAP_val"],
               bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    display.clear_output(wait=True)
    plt.show()

Run test on validation set for each snapshot.

In [ ]:
hists = []
plt.show()
for i, s in enumerate(snapshots):
    print(s)
    weights = os.path.join(SNAP_DIR, s)
    log_file = weights.split('.')[0] + ".log"
    cmd = root+'build/tools/caffe train --solver="{}" --weights="{}"'.format(SOLVER_FILE, weights)
    if len(GPUS)>0:
        cmd += ' --gpu {} 2>&1 | tee {}'.format(GPUS, log_file)
    else:
        cmd += ' 2>&1 | tee {}'.format(log_file)

    # Run the job.
    subprocess.call(cmd, shell=True)
    #plot_scores()

VGG_okutama_SSD_720x576_iter_1000.caffemodel
